<a href="https://colab.research.google.com/github/JohanSH7/SmallBlockchain/blob/main/Task_BLOCKCHAINipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code snippet starts with two important imports:

* **hashlib:** This module is part of Python's standard library and provides hash algorithms. In the context of blockchain, hashing is critical for ensuring the integrity and security of data.
* **json:** This module is used to work with JSON data, which is a common format for structuring data in web applications and blockchains.
* **List (from typing):** This allows you to define data structures that expect a list of certain elements, ensuring that the code is more robust and clear.

**Helper Function:** sha256(data)
This function is a utility function that calculates the SHA-256 hash of the input data. It encodes the data to a string format (UTF-8) and then applies the SHA-256 hashing algorithm.

In [1]:
import hashlib
import json
from typing import List

# Helper function to calculate SHA256 hash
def sha256(data):
    return hashlib.sha256(data.encode('utf-8')).hexdigest()

### **Class: Transaction**

The Transaction class models a basic transaction between two users. It has three key properties:

* **sender:** Represents the public key (address) of the sender. In the context of blockchain, this would correspond to the public key of the wallet making the transaction.
* **receiver:** Represents the public key (address) of the receiver. This is the wallet that will receive the transaction.
* **amount:** The value being transferred between the sender and receiver.

**Methods:**

* **to_dict():** Converts the transaction data into a Python dictionary format. This is useful for easy manipulation and later conversion to JSON, which can be used for creating hashes or saving transaction details.

* **to_string():** Converts the transaction into a JSON string format, which is crucial for calculating the hash of the transaction for inclusion in the Merkle Tree (a fundamental part of blockchain that helps secure the integrity of transactions within blocks). By sorting the keys in the dictionary (sort_keys=True), it ensures consistent hash values regardless of the order in which the fields were added.

In Bitcoin's actual system, each transaction would include additional elements like signatures to verify that the sender is authorized to make the transaction. The current implementation here simplifies this by focusing only on the sender, receiver, and amount.

### **Class: Wallet**
The Wallet class models a user's digital wallet. It contains:

* **public_key:** The public key acts as the wallet's address. In the real blockchain, this key is used for verifying and making transactions.
* **balance:** The wallet's balance, initialized to 0 by default but can be set with an initial value.

**Methods:**

* **update_balance():** A simple method to adjust the balance of the wallet. It adds the given amount to the current balance.

In [2]:
# Class to represent a transaction
class Transaction:
    def __init__(self, sender, receiver, amount):
        self.sender = sender  # public key (address) of sender
        self.receiver = receiver  # public key (address) of receiver
        self.amount = amount

    def to_dict(self):
        return {"sender": self.sender, "receiver": self.receiver, "amount": self.amount}

    # Transaction string to hash for Merkle tree
    def to_string(self):
        return json.dumps(self.to_dict(), sort_keys=True)

# Class to represent a user wallet
class Wallet:
    def __init__(self, public_key, initial_balance=0):
        self.public_key = public_key
        self.balance = initial_balance

    def update_balance(self, amount):
        self.balance += amount

### **The MerkleTree class**
Is responsible for building a Merkle Tree from a list of transactions and generating a Merkle Root—which is a cryptographic hash representing all the transactions in the block.

* **__init__ method:** Takes a list of transactions, computes their hashes, and builds the Merkle root.

* **build_merkle_root method:** Converts the transactions into hashes using SHA-256 and passes them to the compute_merkle_root function.

* **compute_merkle_root method:** Recursively combines adjacent transaction hashes (concatenating and hashing them together) until only a single hash remains—the Merkle Root. If the number of transaction hashes is odd, the last hash is duplicated to ensure pairs can be formed.

In [3]:
# Class to create a Merkle Tree for transactions
class MerkleTree:
    def __init__(self, transactions: List[Transaction]):
        self.transactions = transactions
        self.root = self.build_merkle_root()

    # Create a Merkle root from transaction hashes
    def build_merkle_root(self):
        tx_hashes = [sha256(tx.to_string()) for tx in self.transactions]
        return self.compute_merkle_root(tx_hashes)

    # Compute Merkle root from transaction hashes
    def compute_merkle_root(self, tx_hashes):
        while len(tx_hashes) > 1:
            if len(tx_hashes) % 2 != 0:
                tx_hashes.append(tx_hashes[-1])  # duplicate last hash if odd number
            tx_hashes = [sha256(tx_hashes[i] + tx_hashes[i + 1]) for i in range(0, len(tx_hashes), 2)]
        return tx_hashes[0] if tx_hashes else None

### **Block Class:**
The Block class represents a block in the blockchain, which contains the following attributes:

* **previous_hash:** The hash of the previous block, ensuring each block is linked to the previous one.
* **transactions:** A list of transactions included in this block. These are the operations (e.g., transfers of coins) that will be verified and stored in the block.
* **nonce:** A counter used in the Proof of Work (PoW) mechanism. The correct nonce is found through computational work and is stored in the block.
* **merkle_root:** The root of the Merkle Tree, which is generated from the transactions. It provides a compact way of verifying the integrity of all transactions in the block.

The class also includes:

* **to_dict():** Converts the block's data into a dictionary for hashing or storage purposes.
* **compute_hash():** Uses the SHA-256 algorithm to compute the hash of the block. This hash ensures the integrity of the block and its contents.

### **Blockchain Class:**
The Blockchain class simulates the entire blockchain, composed of multiple blocks. Key attributes include:

* **chain:** A list that holds all the blocks in the blockchain.
* **pending_transactions:** Transactions that are waiting to be added to the next block.
* **difficulty:** The difficulty level for the Proof of Work mechanism, determining how hard it is to mine a new block.

*Key methods include:*

* **create_genesis_block():** Creates the genesis block, the first block in the blockchain. This block is initialized with no transactions and a hash of all zeros.

* **proof_of_work():** Implements the Proof of Work algorithm. It iterates over possible nonce values, recomputing the block's hash until the hash meets the difficulty condition (starting with a specified number of leading zeros). This process simulates the computational effort required to mine a block.

* **add_block():** Adds a new block to the blockchain after successful Proof of Work validation. Once a valid hash is found, the block is added to the chain.

* **validate_transaction():** Ensures that the sender has enough balance to make a transaction. This method verifies whether a wallet has sufficient funds before the transaction is allowed.

* **create_transaction():** Creates a new transaction between two wallets, updating the balance of both the sender and receiver. If the transaction is valid (i.e., the sender has enough funds), it is added to the pool of pending transactions.

* **mine_pending_transactions():** Mines a new block from the list of pending transactions. The miner is rewarded with a coinbase transaction (where the miner is given new coins as a reward for mining the block). After the transactions are mined, they are cleared from the pending list.

In [16]:
# Class to represent a block in the blockchain
class Block:
    def __init__(self, previous_hash, transactions: List[Transaction], nonce=0):
        self.previous_hash = previous_hash  # Hash of the previous block
        self.transactions = transactions  # Block body with transactions
        self.nonce = nonce  # Proof of work nonce
        self.merkle_root = MerkleTree(transactions).root  # Merkle root from transactions

    def to_dict(self):
        return {
            "previous_hash": self.previous_hash,
            "merkle_root": self.merkle_root,
            "nonce": self.nonce,
            "transactions": [tx.to_dict() for tx in self.transactions]
        }

    # Hash of the block
    def compute_hash(self):
        block_string = json.dumps(self.to_dict(), sort_keys=True)
        return sha256(block_string)

# Class to represent the blockchain
class Blockchain:
    MAX_TRANSACTIONS_PER_BLOCK = 4
    def __init__(self, difficulty=2):
        self.chain = []  # List of blocks
        self.pending_transactions = []  # List of pending transactions to be added to blocks
        self.difficulty = difficulty  # Difficulty level for proof-of-work
        self.create_genesis_block()  # Create the genesis block


    # Genesis block (initial block in the chain)
    def create_genesis_block(self):
        genesis_block = Block("0" * 64, [], 0)  # Special genesis block with no transactions
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    # Proof-of-work mechanism: find a nonce that satisfies difficulty condition
    def proof_of_work(self, block):
        block.nonce = 0
        computed_hash = block.compute_hash()
        while not computed_hash.startswith("0" * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash

    # Add a new block to the chain after proof-of-work validation
    def add_block(self, block):
        proof = self.proof_of_work(block)
        block.hash = proof
        self.chain.append(block)

    # Validate transaction: check if sender has enough balance
    def validate_transaction(self, transaction, wallet):
        if transaction.amount <= 0:
            return False  # Sending 0 or less coins is not allowed
        if wallet.balance >= transaction.amount:
            wallet.update_balance(-transaction.amount)
            return True
        else:
            return False

    # Create a new transaction and add it to the pending pool
    def create_transaction(self, sender_wallet, receiver_wallet, amount):
        if self.validate_transaction(Transaction(sender_wallet.public_key, receiver_wallet.public_key, amount), sender_wallet):
            new_transaction = Transaction(sender_wallet.public_key, receiver_wallet.public_key, amount)
            self.pending_transactions.append(new_transaction)
            receiver_wallet.update_balance(amount)
            return new_transaction
        else:
            return None  # Invalid transaction (insufficient funds)

    # Mine a new block with pending transactions
    def mine_pending_transactions(self, miner_wallet: Wallet):
        if not self.pending_transactions:
            print("No hay transacciones pendientes para minar.")
            return None

        # Select a limited number of transactions to include in the block
        transactions_to_include = self.pending_transactions[:self.MAX_TRANSACTIONS_PER_BLOCK]
        previous_hash = self.chain[-1].hash
        block = Block(previous_hash, transactions_to_include)
        self.add_block(block)

        # Remove transactions that have already been mined
        self.pending_transactions = self.pending_transactions[self.MAX_TRANSACTIONS_PER_BLOCK:]

        # Miner reward via coinbase transaction
        coinbase_tx = Transaction(None, miner_wallet.public_key, 50)  # 50 Coins Reward
        self.pending_transactions.append(coinbase_tx)
        miner_wallet.update_balance(50)

    # Method to display blocks in the chain
    def display_chain(self):
        for i, block in enumerate(self.chain):
            print(f"Block {i}:")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Merkle Root: {block.merkle_root}")
            print(f"Nonce: {block.nonce}")
            print(f"Transactions: {block.transactions}")
            print(f"Hash: {block.compute_hash()}")
            print("-" * 50)

Example of use:

In [17]:
if __name__ == "__main__":
    # Create wallets for users
    alice_wallet = Wallet(public_key="alice_public_key", initial_balance=100)
    bob_wallet = Wallet(public_key="bob_public_key", initial_balance=50)
    charlie_wallet = Wallet(public_key="charlie_public_key", initial_balance=75)
    miner_wallet = Wallet(public_key="miner_public_key", initial_balance=0)

    # Create a Blockchain Instance
    blockchain = Blockchain()

    # Create multiple transactions
    blockchain.create_transaction(alice_wallet, bob_wallet, 10)
    blockchain.create_transaction(alice_wallet, charlie_wallet, 5)
    blockchain.create_transaction(bob_wallet, charlie_wallet, 15)
    blockchain.create_transaction(charlie_wallet, alice_wallet, 2)
    blockchain.create_transaction(bob_wallet, alice_wallet, 8)
    blockchain.create_transaction(charlie_wallet, bob_wallet, 3)

    # Mine pending transactions (first time)
    blockchain.mine_pending_transactions(miner_wallet)

    # Create more transactions
    blockchain.create_transaction(alice_wallet, bob_wallet, 20)
    blockchain.create_transaction(bob_wallet, charlie_wallet, 7)

    # Mine pending transactions (second time)
    blockchain.mine_pending_transactions(miner_wallet)

    # Show the blockchain
    blockchain.display_chain()

Block 0:
Previous Hash: 0000000000000000000000000000000000000000000000000000000000000000
Merkle Root: None
Nonce: 0
Transactions: []
Hash: db55bb1f905739fed0728def5b36e3598a4dd986eaa79be4086ecba241464d24
--------------------------------------------------
Block 1:
Previous Hash: db55bb1f905739fed0728def5b36e3598a4dd986eaa79be4086ecba241464d24
Merkle Root: c4a70973de825ae83b831fc5a57fe928bb07eff356b78539f5776f52857d005d
Nonce: 92
Transactions: [<__main__.Transaction object at 0x7c9d758ef010>, <__main__.Transaction object at 0x7c9d758eeec0>, <__main__.Transaction object at 0x7c9d758ee950>, <__main__.Transaction object at 0x7c9d758eecb0>]
Hash: 00455438c8173e39c1c7824fe58e2e9c7c6f69d293fff994633aa520b8219c3d
--------------------------------------------------
Block 2:
Previous Hash: 00455438c8173e39c1c7824fe58e2e9c7c6f69d293fff994633aa520b8219c3d
Merkle Root: 3faa523e7d6959823b05a8cb5e0aaf5b60fe4006d12ca36a934d0b09ac500572
Nonce: 738
Transactions: [<__main__.Transaction object at 0x7c9d758e